In [45]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erfc
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import interactive, Button
#from Diffusion import diffusion_equation
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erfc, erf

class DiffusionModel:
    def __init__(self, matrix_porosity, length, input_conc, input_duration, time_step, t_range_end=250):
        self.matrix_porosity = matrix_porosity
        self.length = length
        self.input_conc = input_conc
        self.input_duration = input_duration
        self.time_step = time_step
        self.t_range_end = t_range_end
        self.reference_lines = []
        self.ref_line_params = []
        self.ref_line_colors = ['black']



    def residence_time(self, length, velocity):
        return length/velocity

    def rel_input_duration(self, input_duration, residence_time):
        return input_duration/residence_time
        
    def positive_pulse(self, matrix_porosity, 
                        pore_diff_coef, 
                        length, 
                        velocity, 
                        a, 
                        t_range, 
                        residence_t):
        
        def t_tr(t,residence_time):
            return t/residence_time
        
        c0 = np.zeros(1*len(t_range))
        i = 0
        
        for ti in t_range:

            if t_tr(ti, residence_t) > 1:

                arg1 = (matrix_porosity*pore_diff_coef * length)/(velocity * a)
                arg2 = np.sqrt(pore_diff_coef * (ti - residence_t))

                c =  erfc(arg1/arg2)
                c0[i] = c
                
                i = i + 1

            else:
                c0[i] = 0
                i = i + 1
                        
        return c0

    def negative_pulse(self, matrix_porosity, 
                        pore_diff_coef, 
                        length, 
                        velocity, 
                        a, 
                        t,
                        rel_input_dur,
                        input_duration,
                        residence_t):
        
        def t_tr(t,residence_time):
            return t/residence_time
        
        c0 = np.zeros(1*len(t))
        i = 0
        
        for ti in t:

            if t_tr(ti, residence_t) > 1 + rel_input_dur:

                arg1 = (matrix_porosity*pore_diff_coef * length)/(velocity * a) 
                arg2 = np.sqrt(pore_diff_coef * (ti - residence_t - input_duration))

                c =  erfc(arg1/arg2)
                c0[i] = c
                
                i = i + 1

            else:
                c0[i] = 0
                i = i + 1
                        
        return c0    

    def relative_concentration(self, matrix_porosity, 
                                   pore_diff_coeff,
                                   length,
                                   velocity,
                                   fracture_aperture,
                                   t_range,
                                   rel_in_dur,
                                   input_duration,
                                   res_t):
            
            positiv_p = self.positive_pulse(matrix_porosity, 
                                            pore_diff_coeff,
                                            length,
                                            velocity,
                                            fracture_aperture,
                                            t_range,
                                            res_t)
            
            negative_p = self.negative_pulse(matrix_porosity, 
                                             pore_diff_coeff,
                                             length,
                                             velocity,
                                             fracture_aperture,
                                             t_range,
                                             rel_in_dur,
                                             input_duration,
                                             res_t)
            
            return positiv_p - negative_p

    def plot_diffusion(self, matrix_porosity, velocity, fracture_aperture, pore_diff_coeff, t_range_end=250):
        self.matrix_porosity = matrix_porosity
        t_range = np.arange(0, t_range_end, self.time_step)
        results = []

        # Calculations
        residence_t = self.residence_time(self.length, velocity)
        rel_in_dur = self.rel_input_duration(self.input_duration, residence_t)
        rel_conc = self.relative_concentration(self.matrix_porosity,
                                               pore_diff_coeff,
                                               self.length,
                                               velocity,
                                               fracture_aperture,
                                               t_range,
                                               rel_in_dur,
                                               self.input_duration,
                                               residence_t)
        results.append(rel_conc)

        # Plotting
        fig, ax = plt.subplots()
        for res, clr in zip(results, ['blue']):
            ax.plot(t_range, res, color=clr, label='Current: Velocity={:.2e}, Aperture={:.2e}, Coeff={:.2e}'.format(velocity, fracture_aperture, pore_diff_coeff))

        # Plot each reference line with formatted parameters in the legend
        for ((t_range_ref, rel_conc_ref), (vel_ref, ap_ref, coeff_ref), color) in zip(self.reference_lines, self.ref_line_params, self.ref_line_colors):
            ax.plot(t_range_ref, rel_conc_ref, color=color, label='Velocity={:.2e}, Aperture={:.2e}, Coeff={:.2e}'.format(vel_ref, ap_ref, coeff_ref))

        ax.set_xlim(0, 30)
        ax.set_ylim(0, 1.1)
        ax.set_xlabel('Time [days]')
        ax.set_ylabel('Relative Concentration [-]')
        ax.legend(loc='upper right', bbox_to_anchor=(2.1, 1.0))
        plt.show()

    def on_button_click(self, button):
        self.add_new_color()

        # Get current slider values
        velocity = self.velocity_slider.value
        fracture_aperture = self.fracture_aperture_slider.value
        pore_diff_coeff = self.pore_diff_coeff_slider.value

        # Use self.t_range_end here
        t_range = np.arange(0, self.t_range_end, self.time_step)
        residence_t = self.residence_time(self.length, velocity)
        rel_in_dur = self.rel_input_duration(self.input_duration, residence_t)
        rel_conc = self.relative_concentration(self.matrix_porosity,
                                               pore_diff_coeff,
                                               self.length,
                                               velocity,
                                               fracture_aperture,
                                               t_range,
                                               rel_in_dur,
                                               self.input_duration,
                                               residence_t)
        
        
        # Store the current line (parameters and results) for future reference
        self.reference_lines.append((t_range, rel_conc))
        self.ref_line_params.append((velocity, fracture_aperture, pore_diff_coeff))

        # Clear output and redraw the plot including reference lines
        clear_output(wait=True)
        self.create_interactive_plot()

    def add_new_color(self):
        colors = ['orange', 'green', 'red', 'purple', 'brown', 'pink']
        for color in colors:
            if color not in self.ref_line_colors:
                self.ref_line_colors.append(color)
                return
        print("All colors used up!")

    def create_interactive_plot(self):
        # Create sliders as class attributes
        # Initialize interactive widgets
        self.matrix_porosity_slider = widgets.FloatSlider(min=0.0, max=0.5, step=0.01, value=0.03, description='Matrix Porosity:')
        self.velocity_slider = widgets.FloatSlider(min=0.1, max=10, step=0.1, value=2, description='Velocity:')
        self.fracture_aperture_slider = widgets.FloatSlider(min=1e-5, max=1e-2, step=1e-5, value=1e-4, description='Fracture Aperture:')
        self.pore_diff_coeff_slider = widgets.FloatSlider(min=1e-8, max=1e-6, step=1e-8, value=2.59e-7, description='Pore Diff Coeff:')

        interactive_plot = interactive(self.plot_diffusion, 
                                    matrix_porosity=self.matrix_porosity_slider,
                                    velocity=self.velocity_slider, 
                                    fracture_aperture=self.fracture_aperture_slider, 
                                    pore_diff_coeff=self.pore_diff_coeff_slider)

        # Button for adding reference lines
        button = Button(description="Add Reference Line")
        button.on_click(lambda b: self.on_button_click(b))

        display(interactive_plot, button)

# Example usage
model = DiffusionModel(matrix_porosity=3e-2, length=10, input_conc=1, input_duration=2, time_step=0.1)
model.create_interactive_plot()


interactive(children=(FloatSlider(value=0.03, description='Matrix Porosity:', max=0.5, step=0.01), FloatSlider…

Button(description='Add Reference Line', style=ButtonStyle())